### From a File


#### Ngrams based

In [1]:
from main import create_taxonomy

filename = "cais_data.csv"
brand_terms = ['cais', 'cais group', 'glas', 'glas funds', 'halo', 'halo investing', 'icapital', 'icapital network']

taxonomy, df, samples = create_taxonomy(filename,
                                        text_column = "keyword",
                                        search_volume_column = "search_volume",
                                        taxonomy_model = "openai", # "palm" or "openai"
                                        use_clustering = False,
                                        ngram_range = (1, 6),
                                        min_df = 5,
                                        brand_terms = brand_terms)

df.to_csv("taxonomy.csv", index=False)

print("\n".join(taxonomy))

c:\Users\jroak\anaconda3\envs\taxonomy\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\jroak\anaconda3\envs\taxonomy\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\jroak\anaconda3\envs\taxonomy\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True

   ngram_size           query  merged_frequency  frequency     score  \
0           1       investing          0.986842   0.625000  0.694132   
1           1           hedge          0.039474   0.176471  0.553986   
2           1     alternative          0.434211   1.000000  0.549265   
3           2  private equity          1.000000   0.676471  0.452186   
4           1      investment          0.355263   0.867647  0.444747   

   search_volume  
0       0.582343  
1       1.000000  
2       0.381424  
3       0.066137  
4       0.278040  


2023-07-06 11:08:48.232 | INFO     | main:create_taxonomy:248 - Reviewing OpenAI's work.
2023-07-06 11:10:31.420 | INFO     | main:create_taxonomy:266 - Getting structure.
2023-07-06 11:10:31.422 | INFO     | main:create_taxonomy:270 - Adding categories.
2023-07-06 11:10:31.425 | INFO     | lib.clustering:fit:427 - Getting embeddings.
2023-07-06 11:10:31.427 | INFO     | lib.clustering:get_embeddings:94 - Using local embeddings
Batches: 100%|██████████| 23/23 [00:02<00:00,  9.44it/s]
2023-07-06 11:10:34.471 | INFO     | lib.clustering:get_cluster_model:177 - Cluster Model: hdbscan
2023-07-06 11:10:34.921 | INFO     | lib.clustering:get_elbow:158 - Using epsilon value: 1.0536712127723509e-08
2023-07-06 11:10:34.923 | INFO     | lib.clustering:fit:433 - Fitting model.
2023-07-06 11:10:34.924 | INFO     | lib.clustering:get_reduced:124 - Reducing embeddings to 5 dims
2023-07-06 11:10:51.172 | INFO     | lib.clustering:fit:436 - Initial Model. Unique Labels: 156
2023-07-06 11:10:51.173 | I

TypeError: only integer scalar arrays can be converted to a scalar index

#### Cluster Description based

##### Using LLM descriptions of clusters

In [ ]:
from main import create_taxonomy, add_categories

filename = "cais_data.csv"
brand_terms = ['cais', 'cais group', 'glas', 'glas funds', 'halo', 'halo investing', 'icapital', 'icapital network']

taxonomy, df, samples = create_taxonomy(filename,
                                        text_column = "keyword",
                                        search_volume_column = "search_volume",
                                        taxonomy_model = "openai", # "palm" or "openai"
                                        use_clustering = True,
                                        use_llm_cluster_descriptions = True,
                                        cluster_embeddings_model = "local", # "palm", "openai", or "local"
                                        min_cluster_size = 5,
                                        min_samples = 2,
                                        use_llm_cluster_descriptions = True,
                                        ngram_range = (1, 6),
                                        min_df = 5,
                                        brand_terms = brand_terms)

df.to_csv("cais_data_taxonomy.csv", index=False)
print("\n".join(taxonomy))

##### Using c-tfidf ngrams for clusters

In [ ]:
from main import create_taxonomy, add_categories

filename = "cais_data.csv"
brand_terms = ['cais', 'cais group', 'glas', 'glas funds', 'halo', 'halo investing', 'icapital', 'icapital network']

taxonomy, df, samples = create_taxonomy(filename,
                                        text_column = "keyword",
                                        search_volume_column = "search_volume",
                                        taxonomy_model = "openai", # "palm" or "openai"
                                        use_clustering = True,
                                        use_llm_cluster_descriptions = False,
                                        cluster_embeddings_model = "local", # "palm", "openai", or "local"
                                        min_cluster_size = 5,
                                        min_samples = 2,
                                        use_llm_cluster_descriptions = True,
                                        ngram_range = (1, 6),
                                        min_df = 5,
                                        brand_terms = brand_terms)

df.to_csv("cais_data_taxonomy.csv", index=False)
print("\n".join(taxonomy))

### From a GSC Account

In [ ]:
from main import create_taxonomy

brand = "Green Group"

taxonomy, df, samples = create_taxonomy("sc-domain:greengroupcompanies.com",
                                        text_column = None,
                                        search_volume_column = None,
                                        platform = "openai", # "palm" or "openai"
                                        days = 30,
                                        ngram_range = (1, 6),
                                        min_df = 2,
                                        brand = None,
                                        limit_queries = 5)

df = add_categories(taxonomy, df, brand) 

df.to_csv("greengroupcompanies_taxonomy.csv", index=False)

df.head()